In [2]:
%load_ext autoreload
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
from ogb.nodeproppred import NodePropPredDataset
import numpy as np
import tensorflow as tf
import tensorflow_gnn as tfgnn
from tensorflow_gnn import runner
from typing import Mapping
from tensorflow_gnn.experimental import sampler
from tensorflow_gnn.models import mt_albis
from tensorflow.keras.callbacks import TensorBoard
import functools
import itertools
from subgraph_dataset_provider import SubgraphDatasetProvider

/media/fastdata/ez/ml/ogb/.env/lib/python3.12/site-packages/outdated/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version
2025-12-02 20:12:28.193727: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
train_sampling_sizes = {
    "bought_together": 8,
}
# graph_schema = tfgnn.read_schema("graph_schema.pbtxt")
dataset = NodePropPredDataset(name = "ogbn-products", root = 'dataset/')
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
NUM_TRAINING_SAMPLES=train_idx.shape[0]
NUM_VALIDATION_SAMPLES = valid_idx.shape[0]
graph, label = dataset[0]
SIZE=2449029

graph_tensor = tfgnn.GraphTensor.from_pieces(
    node_sets={
       "product": tfgnn.NodeSet.from_fields(
           sizes = tf.constant([2449029]),
           features={
               "id": tf.range(0,SIZE),
               "feature": tf.constant(graph["node_feat"]),
               "label": tf.constant(label)
           }
       )
   },
    edge_sets = {
      "bought_together": tfgnn.EdgeSet.from_fields(
          sizes = tf.constant([graph["edge_index"].shape[1]]),
          adjacency = tfgnn.Adjacency.from_indices(
              source = ("product", tf.constant(graph["edge_index"][0,:])),
              target = ("product", tf.constant(graph["edge_index"][1,:])),
          )
      )  
    },
)
valid_ds_provider = SubgraphDatasetProvider(graph_tensor, train_sampling_sizes, tf.data.Dataset.from_tensor_slices(valid_idx), "valid", train_idx.shape[0])

In [8]:
saved_model = tf.saved_model.load(os.path.join("gnn_model", "export"))
signature_fn = saved_model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

def _clean_example_for_serving(graph_tensor):
    label = graph_tensor.node_sets["product"]["label"][0][0].numpy()
    graph_tensor = graph_tensor.remove_features(node_sets={"product": ["label"]})
    serialized_example = tfgnn.write_example(graph_tensor)
    return label, serialized_example.SerializeToString()

num_examples = 10
demo_ds = valid_ds_provider.get_dataset(tf.distribute.InputContext())
dds = itertools.islice(demo_ds, num_examples)
labels, serialized_examples = zip(*[_clean_example_for_serving(gt) for gt in dds])
labels = list(labels)
serialized_examples = list(serialized_examples)
ds = tf.data.Dataset.from_tensor_slices(serialized_examples)
input_dict = {"examples": next(iter(ds.batch(num_examples)))}
output_dict = signature_fn(**input_dict)
logits = output_dict["product_category"]
probabilities = tf.math.softmax(logits).numpy()
classes = probabilities.argmax(axis = 1)
for i, c in enumerate(classes):
    print(f"The predicted class for input {i} is {c:3} "
          f"with predicted probability {probabilities[i, c]:.4}, real values is {labels[i]}")

tf.RaggedTensor(values=Tensor("args_0:0", shape=(None,), dtype=int64), row_splits=Tensor("RaggedFromRowLengths/control_dependency:0", shape=(None,), dtype=int64))
The predicted class for input 0 is   2 with predicted probability 0.9205, real values is 1
The predicted class for input 1 is   6 with predicted probability 0.9995, real values is 6
The predicted class for input 2 is   9 with predicted probability 0.9516, real values is 9
The predicted class for input 3 is   4 with predicted probability 0.9997, real values is 4
The predicted class for input 4 is   7 with predicted probability 0.9987, real values is 7
The predicted class for input 5 is   9 with predicted probability 0.9986, real values is 9
The predicted class for input 6 is   6 with predicted probability 0.6465, real values is 4
The predicted class for input 7 is   1 with predicted probability 0.7109, real values is 1
The predicted class for input 8 is   4 with predicted probability 1.0, real values is 4
The predicted class f

2025-12-02 20:14:26.681150: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
